In [ ]:
import pandas as pd
# NeuralProphet 클래스 불러오기
from neuralprophet import NeuralProphet, set_log_level
from neuralprophet import set_random_seed
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
df = pd.read_csv("인천교통공사_역별일별 이용인원현황_20230630.csv", encoding='cp949')
df

In [ ]:
df=df[df['구분']=='하차']
df.drop(['호선','구분'], axis=1,inplace=True)
df

In [ ]:
df = df[['통행일자', '이용인원', '역명']]
df

In [ ]:
df.columns = ['ds', 'y', 'location']
df['ds'] = pd.to_datetime(df['ds'])
df

In [ ]:
df1 = df[df['location']=='석남']
df1 = df1[['ds','y']]
df1

In [ ]:
# 원본 데이터프레임에서 랜덤하게 10%의 행을 선택하여 새로운 데이터프레임 생성
df1_sampled = df1.sample(frac=0.1, random_state=42)

# 선택한 행을 원본 데이터프레임에서 제거
df1_drop = df1.drop(df1_sampled.index)
df1_sampled = df1_sampled.sort_values(by='ds')

df1_sampled

In [ ]:
df.info()

In [ ]:
plt = df1.plot(x="ds", y="y", figsize=(15, 5))

In [ ]:
# 오류가 아닌 경우 로깅 메시지 비활성화
set_log_level("ERROR")
set_random_seed(0)

# 기본 매개변수로 NeuralProphet 모델 생성
m = NeuralProphet(
#     trend_reg=1, # 트렌드의 기울기(높을수록 기울기가 낮아짐)
    
#     n_changepoints=0,
    # Enable all seasonality components
    yearly_seasonality=True,
    weekly_seasonality=True,
#     daily_seasonality=True,
#     n_lags=10,
#     n_forecasts=30
)

# df1_train, df1_val = m.split_df(df1, valid_p=0.1)
print("Dataset size:", len(df1))
print("Train dataset size:", len(df1_drop))
print("Validation dataset size:", len(df1_sampled))

In [ ]:
# 데이터셋에 모델 피팅하기 (이 과정은 시간이 걸릴 수 있습니다)
metrics = m.fit(df1_drop, validation_df=df1_sampled)
# metrics

# metrics = m.fit(df1)

In [ ]:
forecast = m.predict(df1_sampled)
m.set_plotting_backend("matplotlib")
m.plot(forecast)

In [ ]:
m.plot_parameters(components=["trend", "seasonality"])

In [ ]:
# 검증 데이터에 대한 모델 성능 평가
actual = df1_sampled['y'].values
predicted = forecast['yhat1'].values  # 예측된 값의 열 이름에 따라 조정

mse = mean_squared_error(actual, predicted)
mae = mean_absolute_error(actual, predicted)

print(f"평균 제곱 오차: {mse:.2f}")
print(f"평균 절대 오차: {mae:.2f}")